In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Polygon
from dbfread import DBF
import jenkspy
import json
from jenkspy import jenks_breaks

In [2]:
MAPA_TIPO_ruta = r"C:\Users\ADMIN\OneDrive - Universidad de Guadalajara\3. Voces\2024\Ciudades\Ciudad Guzman\241031 tipologías_ams.gpkg"

#Valor del filtro
filtro = 0

# Define el CRS proyectado que deseas utilizar
CRS_PROYECTADO = 'EPSG:6368' 

In [3]:
# Leer la capa existente del GeoPackage base
MAPA_TIPO_gdf = gpd.read_file(MAPA_TIPO_ruta)

In [4]:
# Define el CRS proyectado que deseas utilizar
CRS_PROYECTADO = 'EPSG:6372' 

# Proyecta el GeoDataFrame a un CRS proyectado
MAPA_TIPO_gdf = MAPA_TIPO_gdf.to_crs(CRS_PROYECTADO)

# Creación de campos y llenado de información
MAPA_TIPO_gdf['area_ha'] = MAPA_TIPO_gdf.area / 10000
MAPA_TIPO_gdf['FILTRADO'] = MAPA_TIPO_gdf['VIVTOT2020'] / MAPA_TIPO_gdf['area_ha']**2
MAPA_TIPO_gdf['DESHABITACION'] = MAPA_TIPO_gdf['VIVPARDES2020'] / MAPA_TIPO_gdf['VIVTOT2020']

In [5]:
MAPA_TIPO_gdf['PROM_DESHAB_MUN'] = MAPA_TIPO_gdf.groupby('CVE_MUN')['DESHABITACION'].transform('mean')


# Calcular las sumas de POPTOT2020 y POPTOT2010 por CVE_MET
sum_poptot2020 = MAPA_TIPO_gdf.groupby('CVE_MUN')['POPTOT2020'].transform('sum')
sum_poptot2010 = MAPA_TIPO_gdf.groupby('CVE_MUN')['POPTOT2010'].transform('sum')

# Calcular TASA_POP_MUN
MAPA_TIPO_gdf['TASA_POP_MUN'] = (((sum_poptot2020 / sum_poptot2010) ** (1/10)) - 1) * 100

In [6]:
# resto de campos
MAPA_TIPO_gdf['DIFPOP'] = MAPA_TIPO_gdf['POPTOT2020'] - MAPA_TIPO_gdf['POPTOT2010']
MAPA_TIPO_gdf['DIFVIV'] = MAPA_TIPO_gdf['VIVTOT2020'] - MAPA_TIPO_gdf['VIVTOT2010']
MAPA_TIPO_gdf['RELPOP'] = MAPA_TIPO_gdf['DIFPOP'] / MAPA_TIPO_gdf['POPTOT2010']
MAPA_TIPO_gdf['RELVIV'] = MAPA_TIPO_gdf['DIFVIV'] / MAPA_TIPO_gdf['VIVTOT2010']
MAPA_TIPO_gdf['CVEPOP'] = np.where(MAPA_TIPO_gdf['RELPOP'] < -0.000000001, 'A', 'B')
MAPA_TIPO_gdf['CVEVIV'] = np.where(MAPA_TIPO_gdf['RELVIV'] < -.0703, '1',
                                   np.where((MAPA_TIPO_gdf['RELVIV'] >= -.0703) & (MAPA_TIPO_gdf['RELVIV'] <= .0703), 
                                            '2', '3'))
MAPA_TIPO_gdf['CVE'] = MAPA_TIPO_gdf['CVEPOP'] + MAPA_TIPO_gdf['CVEVIV'].astype(str)
conditions = [
    (MAPA_TIPO_gdf['POPTOT2020'] == 0),
    (MAPA_TIPO_gdf['POPTOT2010'] == 0) & (MAPA_TIPO_gdf['VIVTOT2010'] == 0) & (MAPA_TIPO_gdf['CVE'] == 'B3')
]
choices = ['DH', 'B3\'']
MAPA_TIPO_gdf['TIPOLOGIA'] = np.select(conditions, choices, default=MAPA_TIPO_gdf['CVE'])

MAPA_TIPO_gdf['DENS_POP10'] = MAPA_TIPO_gdf['POPTOT2010'] / MAPA_TIPO_gdf['area_ha']
MAPA_TIPO_gdf['DENS_POP20'] = MAPA_TIPO_gdf['POPTOT2020'] / MAPA_TIPO_gdf['area_ha']

MAPA_TIPO_gdf['DENS_VIV10'] = MAPA_TIPO_gdf['VIVTOT2010'] / MAPA_TIPO_gdf['area_ha']
MAPA_TIPO_gdf['DENS_VIV20'] = MAPA_TIPO_gdf['VIVTOT2020'] / MAPA_TIPO_gdf['area_ha']

MAPA_TIPO_gdf['CUARTOS2010'] = ((MAPA_TIPO_gdf['VIV1C2010']*1) + (MAPA_TIPO_gdf['VIV2C2010']*2) + (MAPA_TIPO_gdf['VIV3CYM2010']*4))
MAPA_TIPO_gdf['CUARTOS2020'] = ((MAPA_TIPO_gdf['VIV1C2020']*1) + (MAPA_TIPO_gdf['VIV2C2020']*2) + (MAPA_TIPO_gdf['VIV3CYM2020']*4))

MAPA_TIPO_gdf['CPERC2010'] = MAPA_TIPO_gdf['CUARTOS2010'] / MAPA_TIPO_gdf['POPTOT2010']
MAPA_TIPO_gdf['CPERC2020'] = MAPA_TIPO_gdf['CUARTOS2020'] / MAPA_TIPO_gdf['POPTOT2020']

MAPA_TIPO_gdf['HABXVIV2010'] = MAPA_TIPO_gdf['POPTOT2010'] / MAPA_TIPO_gdf['VIVTOT2010']
MAPA_TIPO_gdf['HABXVIV2020'] = MAPA_TIPO_gdf['POPTOT2020'] / MAPA_TIPO_gdf['VIVTOT2020']

# Redondear los cálculos a 3 dígitos
columns_to_round = ['DIFPOP', 'DIFVIV', 'RELPOP', 'RELVIV', 'DENS_POP20', 'DENS_POP10', 'DENS_VIV20', 'DENS_VIV10', 
                    'CUARTOS2010', 'CUARTOS2020', 'CPERC2010', 'CPERC2020', 'HABXVIV2010', 'HABXVIV2020']
MAPA_TIPO_gdf[columns_to_round] = MAPA_TIPO_gdf[columns_to_round].round(3)


# Guardar el GeoPackage base actualizado
MAPA_TIPO_gdf.to_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias_sin_filtro', driver="GPKG")

# Filtrado de información
MAPA_TIPO_gdf_FILTRO = MAPA_TIPO_gdf[MAPA_TIPO_gdf['FILTRADO'] >= filtro]

# Guardar el resultado final
MAPA_TIPO_gdf_FILTRO.to_file(MAPA_TIPO_ruta, layer='Mapa_Tipologias_Filtrado', driver='GPKG')

In [7]:
# Subtipologías
def asignar_subtipologia(row):
    # Valores específicos para deshabitación
    rangos_deshabitacion_grupo = [0, .0703, .1406, .2812]
    
# Asignar subtipología según los rangos definidos
    if rangos_deshabitacion_grupo[0] <= row['DESHABITACION'] < rangos_deshabitacion_grupo[1]:
        return 'a'
    elif rangos_deshabitacion_grupo[1] <= row['DESHABITACION'] < rangos_deshabitacion_grupo[2]:
        return 'b'
    elif rangos_deshabitacion_grupo[2] <= row['DESHABITACION'] < rangos_deshabitacion_grupo[3]:
        return 'c'
    elif row['DESHABITACION'] >= rangos_deshabitacion_grupo[3]:
        return 'd'
    else:
        return '0'  # Valor predeterminado

# Crear un nuevo campo 'R_DESHAB' y llenarlo con las subtipologías asignadas
MAPA_TIPO_gdf_FILTRO['R_DESHAB'] = MAPA_TIPO_gdf_FILTRO.apply(asignar_subtipologia, axis=1)

# Crear el campo SUBTIPOLOGÍA
MAPA_TIPO_gdf_FILTRO['SUBTIPOLOGÍA'] = MAPA_TIPO_gdf_FILTRO['TIPOLOGIA'].astype(str) + '-' + MAPA_TIPO_gdf_FILTRO['R_DESHAB'].astype(str)

# Guardar el GeoPackage con los nuevos campos
MAPA_TIPO_gdf_FILTRO.to_file(MAPA_TIPO_ruta, layer='Mapa_SubTipologias', driver='GPKG')